## Update `rt_scheduled_v_ran/scripts/rt_stop_times.py`
* i think you can go and change the rt_stop_times script to keep the full timestamp, in addition to the seconds. more straightforward to undo the deleting of the columns, instead of trying to recreate the column you want again.
it's possible to turn seconds back, but then the date portion of the datetime needs to be overwritten to be the analysis_date .

In [19]:
import datetime
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

# https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
import sys

sys.path.append("../rt_scheduled_v_ran/scripts")
import rt_stop_times

In [5]:
analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

In [6]:
may_date = "2024-05-22"

In [10]:
scheduled_stop_times = rt_stop_times.prep_scheduled_stop_times(may_date)

In [11]:
scheduled_stop_times.head(2)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key
0,264,351095,0,21600.00,bff13f8993ff18e43577db1f5596e014,4e28c7e213e31a05388f3ba0bdec3d17
1,264,8065945,1,22620.00,bff13f8993ff18e43577db1f5596e014,4e28c7e213e31a05388f3ba0bdec3d17


In [14]:
GTFS_DATA_DICT.rt_stop_times.stage3

'rt_stop_times/stop_arrivals'

In [15]:
def prep_rt_stop_times(
    analysis_date: str
) -> pd.DataFrame: 
    """
    For RT stop arrivals, drop duplicates based on interpolated
    arrival times. Keep the first arrival time,
    the rest would violate a monotonically increasing condition.
    """
    STOP_ARRIVALS = GTFS_DATA_DICT.rt_stop_times.stage3
    
    df = pd.read_parquet(
        f"{SEGMENT_GCS}{STOP_ARRIVALS}_{analysis_date}.parquet"
    ).rename(columns = {"arrival_time": "rt_arrival"})

    df2 = df.sort_values(
        ["trip_instance_key", "stop_sequence"]
    ).drop_duplicates(
        subset=["trip_instance_key", "rt_arrival"]
    ).reset_index(drop=True)
    
    return df2

In [16]:
sched_stop_times = prep_rt_stop_times(may_date)

In [17]:
sched_stop_times.head(2)

,trip_instance_key,stop_sequence,shape_array_key,stop_meters,rt_arrival
0,000054ea29bb07adcfa0fc0f13c4a030,2,51e3d7ca40631da19179eb792af45334,2529.84,2024-05-22 10:16:51
1,000054ea29bb07adcfa0fc0f13c4a030,3,51e3d7ca40631da19179eb792af45334,3043.91,2024-05-22 10:18:35
